In [1]:
import tensorflow as tf
# import tensorflow.contrib.eager as tfe
# tf.tile(gfc_3, multiples=[1, 4, 4, 1], name='f_1')

In [2]:
def create_conv2d(inputs, filters, strides, reuse, name):
    return tf.layers.conv2d(inputs=inputs,
                           filters=filters,
                           kernel_size=3,
                           strides=strides,
                           padding='same',
                           reuse=reuse,
                           name=name)

In [3]:
def create_dense(inputs, units, name):
    return tf.layers.dense(inputs=inputs,
                           units=units,
                           activation=tf.nn.leaky_relu,
                           name=name)

In [4]:
tf.reset_default_graph()

h, w = 128, 128
img = tf.placeholder(dtype=tf.float32, shape=[None, h, w, 1])
img_scaled = tf.placeholder(dtype=tf.float32, shape=[None, 112, 112, 1])
# batch_size = 50
# img = tf.contrib.eager.Variable(tf.random_uniform(shape=[batch_size, h, w, 1]))
# img_scaled = tf.contrib.eager.Variable(tf.random_uniform(shape=[batch_size, 112, 112, 1]))

# -----------------------------LOW-LEVEL FEATURES: FIXED-----------------------------
fconv2d_11 = create_conv2d(inputs=img_scaled, filters=64, strides=2, reuse=None, name='lf_conv2d_11')
fconv2d_12 = create_conv2d(inputs=fconv2d_11, filters=128, strides=1, reuse=None, name='lf_conv2d_12')

fconv2d_21 = create_conv2d(inputs=fconv2d_12, filters=128, strides=2, reuse=None, name='lf_conv2d_21')
fconv2d_22 = create_conv2d(inputs=fconv2d_21, filters=256, strides=1,  reuse=None, name='lf_conv2d_22')

fconv2d_31 = create_conv2d(inputs=fconv2d_22, filters=256, strides=2, reuse=None, name='lf_conv2d_31')
fconv2d_32 = create_conv2d(inputs=fconv2d_31, filters=512, strides=1, reuse=None, name='lf_conv2d_32')

# -----------------------------GLOBAL FEATURE NETWORK-----------------------------
gconv2d_1 = create_conv2d(inputs=fconv2d_32, filters=512, strides=2, reuse=None, name='gf_conv2d_1')
gconv2d_2 = create_conv2d(inputs=gconv2d_1, filters=512, strides=1, reuse=None, name='gf_conv2d_2')
gconv2d_3 = create_conv2d(inputs=gconv2d_2, filters=512, strides=2, reuse=None, name='gf_conv2d_3')
gconv2d_4 = create_conv2d(inputs=gconv2d_3, filters=512, strides=1, reuse=None, name='gf_conv2d_4')

gfc_1 = create_dense(inputs=gconv2d_4, units=1024, name='gf_fc_1')
gfc_2 = create_dense(inputs=gfc_1, units=512, name='gf_fc_2')
gfc_3 = create_dense(inputs=gfc_2, units=256, name='gf_fc_3')

# -----------------------------LOW-LEVEL FEATURES: VARIABLE-----------------------------
vconv2d_11 = create_conv2d(inputs=img, filters=64, strides=2, reuse=True, name='lf_conv2d_11')
vconv2d_12 = create_conv2d(inputs=vconv2d_11, filters=128, strides=1, reuse=True, name='lf_conv2d_12')

vconv2d_21 = create_conv2d(inputs=vconv2d_12, filters=128, strides=2, reuse=True, name='lf_conv2d_21')
vconv2d_22 = create_conv2d(inputs=vconv2d_21, filters=256, strides=1, reuse=True, name='lf_conv2d_22')

vconv2d_31 = create_conv2d(inputs=vconv2d_22, filters=256, strides=2, reuse=True, name='lf_conv2d_31')
vconv2d_32 = create_conv2d(inputs=vconv2d_31, filters=512, strides=1, reuse=True, name='lf_conv2d_32')

# -----------------------------MID-LEVEL FEATURE NETWORK-----------------------------
mconv2d_1 = create_conv2d(inputs=vconv2d_32, filters=512, strides=1, reuse=None, name='mf_conv2d_1')
mconv2d_2 = create_conv2d(inputs=mconv2d_1, filters=256, strides=1, reuse=None, name='mf_conv2d_2')

# -----------------------------FUSION NETWORK-----------------------------
fusion = tf.concat([mconv2d_2, tf.tile(gfc_3, multiples=[1, 4, 4, 1])], axis=3, name='fusion')

In [5]:
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    file_writer = tf.summary.FileWriter('logs', sess.graph)